# OSM Geohash File Combination

In [1]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter
from string import ascii_uppercase

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-21 13:55:04.372389


In [2]:
# Global Params
basedir="/Users/tgadf/Downloads/osm"
prec=7
mainkey="OSM"
letters=ascii_uppercase

# Combine Geohash Data

In [4]:
import pickle
prec=7
from glob import glob
from os.path import basename,splitext, join
from collections import Counter

#mtypes   = set([splitext(basename(x))[0].split('-')[1] for x in vals])
mtypes  = ["traffic", "pofw", "pois", "transport", "buildings"]
mtypes  = ["buildings"]
mtypes  = ["pofw"]
mtypes  = ["traffic"]
mtypes  = ["transport"]
mtypes  = ["pois"]
mtypes  = ["traffic", "pofw", "pois", "transport"]
mtypes  = ["water"]
mtypes  = ["landuse"]
#mtypes  = ["place"]
#mtypes  = ["natural"]

print("Mtypes --> {0}".format(mtypes))
for mtype in mtypes:
    if mtype in ["pofw","traffic","transport","pois","water","natural","place","landuse"]:
        vals    = glob(join(basedir, "*-{0}-{1}-geos.p".format(mtype, prec)))        
        states  = set([splitext(basename(x))[0].split('-{0}'.format(mtype))[0] for x in vals])
    else:
        vals    = glob(join(basedir, "*-{0}-geos.p".format(prec)))
        states  = set([splitext(basename(x))[0].split('-gis_osm')[0] for x in vals])
    print("States --> {0}".format(states))

    print("============== {0} ==============".format(mtype))
    if mtype in ["water", "landuse"]:
        fnames = letters[:5]
    elif mtype == "place":
        fnames = letters[:3]
    else:
        fnames = letters[:1]
    for let in range(len(fnames)):
        ngeosMtypes = 0
        ngeos = 0
        geomap  = {}
        records = {}
        cntr    = Counter()
        for istate,state in enumerate(states):
            if istate % len(fnames) != let:
                continue
            base = "{0}*{1}*{2}".format(state, mtype, prec)
            recfiles = glob(join(basedir, "{0}-data.p".format(base)))
            geofiles = glob(join(basedir, "{0}-geos.p".format(base)))

            #print("  Found {0} and {1} rec/geo files for {2}".format(len(recfiles), len(geofiles), mtype))
            for i in range(len(geofiles)):
                recfile = recfiles[i]
                geofile = geofiles[i]

                try:
                    recdata = pickle.load(open(recfile, "rb"))
                    geodata = pickle.load(open(geofile, "rb"))
                    #print("Opened {0}".format(recfile))
                    #print("Opened {0}".format(geofile))
                except:
                    print("Could not open {0}".format(geofile))
                    1/0
                    continue

                #print(geodata.keys())
                #continue
                ngeos = 0
                for geoid,geos in geodata.items():
                    ngeos += len(geos)
                    ngeosMtypes += len(geos)
                    for geo in geos:
                        if geomap.get(geo) is None:
                            geomap[geo] = Counter()
                        #print(geo,mtype,geoid)
                        #print(geoid)
                        #print(recdata.keys())
                        if recdata.get(geoid) is not None:
                            try:
                                #print(geoid)
                                #print(recdata[geoid])
                                #name = recdata[geoid]['Name']
                                #print(name)
                                name = geoid
                            except:
                                name = geoid
                                #print(name)
                        else:
                            name = geoid
                        cntr[name] += 1
                        geomap[geo][name] += 1

                for geoid,rec in recdata.items():
                    records[geoid] = rec['Name']
                    break

            print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,state,ngeos,ngeosMtypes,len(geomap)))            
        print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,"","",ngeosMtypes,len(geomap)))

        for k,v in cntr.most_common():
            k = str(k)
            if k.find(" ") != -1 or k.find(";") != -1 or k.find("_") != -1 or k.find(",") != -1 or k[0].isupper():
                print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                found = True
                    
        for geo in geomap.keys():
            geomap[geo] = dict(geomap[geo])
        

        if len(fnames) == 1:
            fname="geomap-{0}-{1}OSM.p".format(prec, mtype)
        else:
            fname="geomap-{0}-{1}{2}OSM.p".format(prec, mtype, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(geomap,  open(fname, "wb"))

        if len(fnames) == 1:
            fname="georec-{0}-{1}OSM.p".format(prec, mtype)
        else:
            fname="georec-{0}-{1}{2}OSM.p".format(prec, mtype, fnames[let])
        print("Writing {0}".format(fname))
        pickle.dump(records,  open(fname, "wb"))

Mtypes --> ['landuse']
States --> {'district-of-columbia', 'colorado', 'kansas', 'massachusetts', 'new-york', 'arkansas', 'new-mexico', 'iowa', 'minnesota', 'california', 'michigan', 'louisiana', 'vermont', 'alaska', 'ohio', 'rhode-island', 'illinois', 'oklahoma', 'tennessee', 'mississippi', 'texas', 'georgia', 'nebraska', 'connecticut', 'indiana', 'delaware', 'west-virginia', 'pennsylvania', 'virginia', 'utah', 'new-jersey', 'wyoming', 'missouri', 'florida', 'hawaii', 'south-dakota', 'alabama', 'arizona', 'maryland', 'north-dakota', 'puerto-rico', 'kentucky', 'nevada', 'idaho', 'montana', 'south-carolina', 'north-carolina', 'wisconsin', 'oregon', 'new-hampshire', 'maine', 'washington'}
============== landuse ==============
landuse             	district-of-columbia	16670       	16670       	---> 13756 <---
landuse             	arkansas            	40872       	57542       	---> 52235 <---
landuse             	michigan            	423958      	481500      	---> 445645 <---
landuse      